In [1]:
!pip install imutils

Defaulting to user installation because normal site-packages is not writeable


In [2]:

import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K


In [3]:
def load(paths, verbose=-1):
    '''expects images for each class in seperate dir, 
    e.g all digits in 0 class in the directory named 0 '''
    data = list()
    labels = list()
    # loop over the input images
    for (i, imgpath) in enumerate(paths):
        # load the image and extract the class labels
        im_gray = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
        im_gray=cv2.resize(im_gray,(160,160))
        image = np.array(im_gray).flatten()
        label = imgpath.split(os.path.sep)[-2]
        # scale the image to [0, 1] and add to list
        data.append(image/255)
        labels.append(label)
        # show an update every `verbose` images
        if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
            print("[INFO] processed {}/{}".format(i + 1, len(paths)))
    # return a tuple of the data and labels
    return data, labels

In [4]:
#declear path to your mnist data folder
img_path = 'DATASET'

#get the path list using the path object
image_paths = list(paths.list_images(img_path))

#apply our function
image_list, label_list = load(image_paths, verbose=10000)

#binarize the labels
lb = LabelBinarizer()
label_list = lb.fit_transform(label_list)

#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(image_list, 
                                                    label_list, 
                                                    test_size=0.1, 
                                                    random_state=42)

In [5]:
len(image_list)

832

In [6]:
len(label_list)

832

In [7]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):


   
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    data = list(zip(image_list, label_list))
    random.shuffle(data)

    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

In [8]:
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

In [9]:
clients

{'client_1': [(array([1., 1., 1., ..., 1., 1., 1.]), array([0, 1, 0, 0])),
  (array([0.18431373, 0.05882353, 0.0627451 , ..., 0.03529412, 0.01568627,
          0.01960784]),
   array([0, 1, 0, 0])),
  (array([0.99215686, 0.05490196, 0.09803922, ..., 1.        , 0.99607843,
          0.99215686]),
   array([0, 0, 0, 1])),
  (array([0.29019608, 0.19215686, 0.        , ..., 0.00392157, 0.03529412,
          0.02745098]),
   array([0, 0, 0, 1])),
  (array([0.03529412, 0.01960784, 0.03921569, ..., 0.01176471, 0.00392157,
          0.        ]),
   array([1, 0, 0, 0])),
  (array([0.14901961, 0.13333333, 0.0745098 , ..., 0.00392157, 0.01568627,
          0.00392157]),
   array([0, 0, 0, 1])),
  (array([0.43921569, 0.44313725, 0.43921569, ..., 0.03137255, 0.04313725,
          0.04705882]),
   array([0, 0, 1, 0])),
  (array([0.        , 0.        , 0.        , ..., 0.00392157, 0.03137255,
          0.00392157]),
   array([0, 1, 0, 0])),
  (array([1.        , 0.98431373, 0.99607843, ..., 0.0588

In [10]:
def batch_data(data_shard, bs=32):

    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [11]:
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(tf.ragged.constant(data))
    

test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [12]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(25600,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

In [13]:
lr = 0.01 
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )        

C:\Users\jains\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [14]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
   
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
   
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
 
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
  
    avg_grad = list()
  
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
 
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [15]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(784, 4)
        

for comm_round in range(comms_round):
            
 
    global_weights = global_model.get_weights()
  
    scaled_local_weight_list = list()

   
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
   
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784, 4)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)

        local_model.set_weights(global_weights)
        
       
        local_model.fit(clients_batched[client], epochs=3, verbose=0)
        
       
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        K.clear_session()
        
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    global_model.set_weights(average_weights)

    
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

3/3 [==============================] - 0s 6ms/step
comm_round: 0 | global_acc: 40.476% | global_loss: 1.3583686351776123
3/3 [==============================] - 0s 6ms/step
comm_round: 1 | global_acc: 41.667% | global_loss: 1.3145051002502441
3/3 [==============================] - 0s 9ms/step
comm_round: 2 | global_acc: 58.333% | global_loss: 1.2911036014556885
3/3 [==============================] - 0s 6ms/step
comm_round: 3 | global_acc: 53.571% | global_loss: 1.2364603281021118
3/3 [==============================] - 0s 8ms/step
comm_round: 4 | global_acc: 58.333% | global_loss: 1.2354830503463745
3/3 [==============================] - 0s 6ms/step
comm_round: 5 | global_acc: 64.286% | global_loss: 1.1926079988479614
3/3 [==============================] - 0s 8ms/step
comm_round: 6 | global_acc: 61.905% | global_loss: 1.1751922369003296
3/3 [==============================] - 0s 7ms/step
comm_round: 7 | global_acc: 65.476% | global_loss: 1.1535711288452148
3/3 [===========================

3/3 [==============================] - 0s 7ms/step
comm_round: 67 | global_acc: 78.571% | global_loss: 0.9482477903366089
3/3 [==============================] - 0s 7ms/step
comm_round: 68 | global_acc: 78.571% | global_loss: 0.9483562707901001
3/3 [==============================] - 0s 7ms/step
comm_round: 69 | global_acc: 78.571% | global_loss: 0.9485734105110168
3/3 [==============================] - 0s 7ms/step
comm_round: 70 | global_acc: 78.571% | global_loss: 0.9481164813041687
3/3 [==============================] - 0s 7ms/step
comm_round: 71 | global_acc: 78.571% | global_loss: 0.9481327533721924
3/3 [==============================] - 0s 7ms/step
comm_round: 72 | global_acc: 78.571% | global_loss: 0.9482845067977905
3/3 [==============================] - 0s 7ms/step
comm_round: 73 | global_acc: 78.571% | global_loss: 0.9479529857635498
3/3 [==============================] - 0s 6ms/step
comm_round: 74 | global_acc: 78.571% | global_loss: 0.9477335810661316
3/3 [===================

In [16]:
def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss